<a href="https://colab.research.google.com/github/Viaxiz/VariesDemo/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 导入相关包
# 测试环境 K80 pytorch1.10
import torch
import torchvision 
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
# 查看pytorch版本和gpu是否可用
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu116
True


In [4]:
# 如果网络能在GPU中训练，就使用GPU；否则使用CPU进行训练
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# 这个函数包括了两个操作：将图片转换为张量，以及将图片进行归一化处理
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(mean = [0.5],std = [0.5])])
                                
# 设置了每个包中的图片数据个数
BATCH_SIZE = 64
EPOCHS = 10

In [6]:
from torchvision import datasets, transforms

In [7]:
data_train = datasets.MNIST(root = "./data/",
                            transform=transform,
                            train = True,
                            download = True)
# root 指代文件保存位置
data_test = datasets.MNIST(root="./data/",
                           transform = transform,
                           train = False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [8]:
# 建立一个数据迭代器
# 装载训练集
train_loader = torch.utils.data.DataLoader(dataset=data_train,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
# 装载测试集
test_loader = torch.utils.data.DataLoader(dataset=data_test,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)
# batch_size设置了每批装载的数据图片为64个，shuffle设置为True在装载过程中为随机乱序

下面构建数据训练模型并创建实例

In [9]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(stride=2,kernel_size=2))
        self.dense = torch.nn.Sequential(torch.nn.Linear(14*14*128,1024),
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=0.5),
                                         torch.nn.Linear(1024, 10))
    def forward(self, x):
        x = self.conv1(x)
        x = x.view(-1, 14*14*128)
        x = self.dense(x)
        return x

# 完成数据装载后就可以构建核心程序了，这里构建的是一个包含了卷积层和全连接层的神经网络，其中卷积层使用torch.nn.Conv2d来构建，
# 激活层使用torch.nn.ReLU来构建，池化层使用torch.nn.MaxPool2d来构建，全连接层使用torch.nn.Linear来构建

# 其中定义了torch.nn.Dropout(p=0.5)防止模型的过拟合

# forward函数定义了前向传播，其实就是正常卷积路径。首先经过self.conv1(x)卷积处理，然后进行x.view(-1, 14*14*128)压缩扁平化处理，最后通过self.dense(x)全连接进行分类

# 之后就是对Model对象进行调用，然后定义loss计算使用交叉熵，优化计算使用Adam自动化方式，最后就可以开始训练了


In [14]:
# 在训练前可以查看神经网络架构了，print输出显示如下
print(Model())

Model(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [10]:
# 开始训练
model = Model()
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [12]:
print(model)

Model(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10, bias=True)
  )
)
